# EasyGen

EasyGen is a visual user interface to help set up simple neural network generation tasks.

There are a number of neural network frameworks (Tensorflow, TFLearn, Keras, PyTorch, etc.) that implement standard algorithms for generating text (e.g., recurrent neural networks, sequence2sequence) and images (e.g., generative adversarial networks). They require some familairity with coding. Beyond that, just running a simple experiment may require a complex set of steps to clean and prepare the data.

EasyGen allows one to quickly set up the data cleaning and neural network training pipeline using a graphical user interface and a number of self-contained "modules" that implement stardard data preparation routines. EasyGen differs from other neural network user interfaces in that it doesn't focus on the graphical instantiation of the neural network itself. Instead, it provides an easy to use way to instantiate some of the most common neural network algorithms used for generation. EasyGen focuses on the data preparation.

For documentation see the [EasyGen Github repo](https://https://github.com/markriedl/easygen)

To get started:

1. Clone the [EasyGen notebook](https://drive.google.com/open?id=1XNiOuNtMnItl5CPGvRjEvj9C78nDuvXj) by following the link and selecting File -> Save a copy in Drive.

2. Turn on GPU support under Edit -> Notebook setting.

3. Run the cells in Sections 1. Some are optional if you know you aren't going to be using particular features.

4. Run the cell in Section 2. If you know there are any models or datasets that you won't be using you can skip them.

5. Run the cell in Section 3. This creates a blank area below the cell in which you can use the buttons to create your visual program. An example program is loaded automatically. You can clear it with the "clear" button below it. Afterwards you can create your own programs. Selecting "Make New Module" will cause the new module appears graphically above and can be dragged around. The inputs and outputs of different modules can be connected together by clicking on an output (red) and dragging to an input (green). Gray boxes are parameters that can be edited. Clicking on a gray box causes a text input field to appear at the bottom of the editing area, just above the "Make New Module" controls.

6. Save your program by entering a program name and pressing the "save" button.

7. Run your program by editing the program name in the cell in Section 4 and then running the cell.

# 1. Setup

Download EasyGen

In [1]:
!git clone https://github.com/markriedl/easygen.git
!cp easygen/*.js /usr/local/share/jupyter/nbextensions/google.colab/
!cp easygen/images/*.png /usr/local/share/jupyter/nbextensions/google.colab/

fatal: destination path 'easygen' already exists and is not an empty directory.


Install requirements

In [0]:
!apt-get update
!apt-get install chromium-chromedriver
!pip install -r easygen/requirements.txt

Download StyleGAN

In [0]:
!git clone https://github.com/NVlabs/stylegan.git
!cp easygen/stylegan_runner.py stylegan

Download GPT-2

In [0]:
!git clone https://github.com/nshepperd/gpt-2

Create backend hooks for saving and loading programs

In [0]:
import IPython
from google.colab import output

def python_save_hook(file_text, filename):
  import easygen
  import hooks
  status = hooks.python_save_hook_aux(file_text, filename)
  ret_status = 'true' if status else 'false'
  return IPython.display.JSON({'result': ret_status})

def python_load_hook(filename):
  import easygen
  import hooks
  result = hooks.python_load_hook_aux(filename)
  return IPython.display.JSON({'result': result})

def python_cwd_hook(dir):
  import easygen
  import hooks
  result = hooks.python_cwd_hook_aux(dir)
  return IPython.display.JSON({'result': result})

def python_copy_hook(path1, path2):
  import easygen
  import hooks
  status = hooks.python_copy_hook_aux(path1, path2)
  ret_status = 'true' if status else 'false'
  return IPython.display.JSON({'result': ret_status})

def python_move_hook(path1, path2):
  import easygen
  import hooks
  status = hooks.python_move_hook_aux(path1, path2)
  ret_status = 'true' if status else 'false'
  return IPython.display.JSON({'result': ret_status})

def python_open_text_hook(path):
  import easygen
  import hooks
  status = hooks.python_open_text_hook_aux(path)
  ret_status = 'true' if status else 'false'
  return IPython.display.JSON({'result': ret_status})
    
def python_open_image_hook(path):
  import easygen
  import hooks
  status = hooks.python_open_image_hook_aux(path)
  ret_status = 'true' if status else 'false'
  return IPython.display.JSON({'result': ret_status})

def python_mkdir_hook(path, dir_name):
  import easygen
  import hooks
  status = hooks.python_mkdir_hook_aux(path, dir_name)
  ret_status = 'true' if status else 'false'
  return IPython.display.JSON({'result': ret_status})

def python_trash_hook(path):
  import easygen
  import hooks
  status = hooks.python_trash_hook_aux(path)
  ret_status = 'true' if status else 'false'
  return IPython.display.JSON({'result': ret_status})

def python_run_hook(path):
  import easygen
  program_file_name = path
  easygen.main(program_file_name)
  return IPython.display.JSON({'result': 'true'})

output.register_callback('notebook.python_cwd_hook', python_cwd_hook)
output.register_callback('notebook.python_copy_hook', python_copy_hook)
output.register_callback('notebook.python_move_hook', python_move_hook)
output.register_callback('notebook.python_open_text_hook', python_open_text_hook)
output.register_callback('notebook.python_open_image_hook', python_open_image_hook)
output.register_callback('notebook.python_save_hook', python_save_hook)
output.register_callback('notebook.python_load_hook', python_load_hook)
output.register_callback('notebook.python_mkdir_hook', python_mkdir_hook)
output.register_callback('notebook.python_trash_hook', python_trash_hook)
output.register_callback('notebook.python_run_hook', python_run_hook)

Import EasyGen

In [0]:
import sys
sys.path.insert(0, 'easygen')
import easygen

# 2. Download pre-trained neural network models

## 2.1 Download GPT-2 and StyleGan models

Download the GPT-2 small 117M model. Will save to ```models/117M``` directory.


In [0]:
!python gpt-2/download_model.py 117M

Download the GPT-2 medium 345M model. Will save to ```models/345M``` directory.

In [0]:
!python gpt-2/download_model.py 345M

Download the StyleGAN cats model (256x256). Will save as "cats256x256.pkl" in the home directory.

In [0]:
!wget -O cats256x256.pkl https://www.dropbox.com/s/1w97383h0nrj4ea/karras2019stylegan-cats-256x256.pkl?dl=0

## 2.2 Download Wikipedia

You only need to do this if you are using the ```ReadWikipedia``` functionality. This takes a long time. You may want to skip it if you know you wont be scraping data from Wikipedia.

In [0]:
!wget -O wiki.zip https://www.dropbox.com/s/39w6mj1akwy2a0r/wiki.zip?dl=0
!unzip wiki.zip

# 3. Run the GUI

Run the cell below. This will load a default example program that generate new, fictional paint names. Use the "clear" button to clear it and make your own.

When done, name the program and press the "save" button. You should see your file appear in the file listing in the left panel.

In [0]:
%%html
<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1.0"/>

<style>
// MAKE CANVAS 
canvas {
    border:1px solid #d3d3d3;
    background-color: #f1f1f1;
}
</style>
</head>
<body>
<script src="nbextensions/google.colab/module_dicts.js"></script>
<script src="nbextensions/google.colab/gui.js"></script>
<script>load_program("easygen/examples/make_new_colors")</script>
<div id="inp">
  <h1 id="inp_module">text</h1>
  <strong id="inp_param">text</strong>
  <input id="inp_val" />
  <button onmouseup="do_input_button_up()">ok</button>
</div>
<div id="make">
  <h1>Make New Module</h1>
  <select id="module_select"></select>
  <button onmouseup="do_make_module_button_up()">Add Module</button>
</div>
<div>
  <h1>Save Program</h1>
  <input id="inp_save" />
  <button onmouseup="save_program()">Save</button>
</div>
<div>
  <h1>Load Program</h1>
  <input id="inp_load" />
  <button onmouseup="load_program()">Load</button>
</div>
<div>
  <h1>Clear Program</h1>
  <button onmouseup="clear_program()">Clear</button>
</div>
</body>
</html>

# 4. Run Your Program

This will run a default example program that will generate new, fictional paint names. If you don't want to run that program, skip to the next cell.

In [0]:
program_file_name = 'easygen/examples/make_new_colors'
easygen.main(program_file_name)

Once you've made your own program, run the cell below, enter the program below, and the press the run button.


In [0]:
%%html
<html>
<body>
<script src="nbextensions/google.colab/run_program.js"></script>
</script>
<b>Run Program:</b> <input id="inp_run" /> <button onmouseup="run_program()">Run</button>  
</body>
</html>

# 5. View Your Output Files

Run the cell below to launch a file manager so you can view your text and image files. 

You can use the panel to the left to download any files written to disk.

In [0]:
%%html
<html>
<body>
<script src="nbextensions/google.colab/file_manager.js"></script>
<h1>Manage Files</h1>
<table cols="3" border="0">
  <tr><td><strong id="path1">/content</strong></td><td></td><td><strong id="path2">/content</strong></td></tr>
  <tr><td><select multiple id="file_list1"></select></td><td><p><button id="copy_button" onmouseup="do_copy_mouse_up()">Copy --></button></p><p><button id="move_button" onmouseup="do_move_mouse_up()">Move --></button></p></td><td><select multiple id="file_list2"></select></td></tr>
  <tr><td><button id="open_text_button" onmouseup="do_open_text_mouse_up()">Open Text</button><br><button id="open_image_button" onmouseup="do_open_image_mouse_up()">Open Image</button><br><button id="open_text_button" onmouseup="do_trash_mouse_up()">Send to trash</button></td></tr>
  </table>
  <h2>Make Directories</h2>
<input id="mkdir_input" /><button onmouseup="do_mkdir_mouse_up()">Make directory</button>
</body>
</html>